In [ ]:
import pandas as pd
import pyodbc 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sqlalchemy as sal
from sqlalchemy import create_engine

In [ ]:
#Connect to Data Structure for sqlalchemy is: "mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server" --change RPC1 for other volumes

RPC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RPC1.connect()

RC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RC1.connect()


In [ ]:
RPC1RibbonInputDF = pd.DataFrame()
RC1RibbonInputDF = pd.DataFrame()

In [ ]:
RC1Aiis= ["514", "3679", "2610", "410"]
RPC1Aiis = ["192", "262", "265", "1685", "2710"]

In [ ]:
#Pull Data from RPC1
for x in RPC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.RibbonInputs @TargetCell= " + "'" + x + "'"
    RI = pd.read_sql(query, con = RPC1)
    RPC1RibbonInputDF = RPC1RibbonInputDF.append(RI)

RPC1RibbonInputDF

In [ ]:
RPC1RibbonInputDF.SourceParentLabel.value_counts()

In [ ]:
RPC1RibbonInputDF = RPC1RibbonInputDF.reset_index(drop = True)

In [ ]:
#Pull data from RC1
for x in RC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.RibbonInputs @TargetCell= " + "'" + x + "'"
    RI = pd.read_sql(query, con = RC1)
    RC1RibbonInputDF = RC1RibbonInputDF.append(RI)

RC1RibbonInputDF

In [ ]:
#Find all Labels
RC1RibbonInputDF.SourceParentLabel.value_counts()

In [ ]:
#Fix any odd names in RC1 DF or simplify
RC1RibbonInputDF.replace('CBbwf', "CBb", inplace=True)

In [ ]:
#Make comparison bargraphs
f, ax = plt.subplots()
sns.countplot(data = RPC1RibbonInputDF, x ='SourceParentLabel', order=RPC1RibbonInputDF.SourceParentLabel.value_counts().index, hue="TargetParent")

handles, labels = ax.get_legend_handles_labels()
ax.set_title('RPC1 Aii Ribbon Inputs')
ax.legend( title="Aii Target",
          handletextpad=0, columnspacing=1,
          loc="upper right",  frameon=True)

g, ax2 = plt.subplots()
sns.countplot(data = RC1RibbonInputDF, x ='SourceParentLabel', order=RC1RibbonInputDF.SourceParentLabel.value_counts().index, hue="TargetParent")

handles, labels = ax.get_legend_handles_labels()
ax2.set_title('RC1 Aii Ribbon Inputs')
ax2.legend( title="Aii Target",
          handletextpad=0, columnspacing=1,
          loc="upper right",  frameon=True)

In [ ]:
#create simplified count-based DFs
SimpRPC1RibbonInputDF = RPC1RibbonInputDF.groupby("TargetParent")["SourceParentLabel"].value_counts().to_frame(name = 'counts').reset_index()
SimpRC1RibbonInputDF = RC1RibbonInputDF.groupby("TargetParent")["SourceParentLabel"].value_counts().to_frame(name = 'counts').reset_index()


In [ ]:
SimpRPC1RibbonInputDF

In [ ]:
df = SimpRPC1RibbonInputDF
df["TotalInput"] = df.groupby('TargetParent')["counts"].transform('sum')
df["PercentofInput"] = df["counts"] / df["TotalInput"]*100
df["Volume"]= 'RPC1'

df

In [ ]:
df = SimpRC1RibbonInputDF
df["TotalInput"] = df.groupby('TargetParent')["counts"].transform('sum')
df["PercentofInput"] = df["counts"] / df["TotalInput"]*100
df["Volume"]= 'RC1'

df

In [ ]:
frames = [SimpRC1RibbonInputDF, SimpRPC1RibbonInputDF]
CombinedRI = pd.concat(frames).reset_index()

CombinedRI

In [ ]:
CombinedRI = CombinedRI.rename({'counts':'TotalRibbonCount'}, axis = 'columns')

In [ ]:
CombinedRI

In [ ]:
sns.relplot(x="TotalRibbonCount", y="PercentofInput", hue="Volume", style="SourceParentLabel", data=CombinedRI);

In [ ]:

RPC1 = CombinedRI.loc[CombinedRI.Volume=='RPC1']
RC1 = CombinedRI.loc[CombinedRI.Volume=='RC1']
RodBC = CombinedRI.loc[CombinedRI.SourceParentLabel=='RodBC']
CBa = CombinedRI.loc[CombinedRI.SourceParentLabel=='CBa']
CBb = CombinedRI.loc[CombinedRI.SourceParentLabel=='CBb']

sns.displot(CombinedRI, x=RodBC.TotalRibbonCount, y=RodBC.PercentofInput, hue="Volume", kind = "kde", fill = True).set(title = "RodBC Distributions")
sns.displot(CombinedRI, x=CBa.TotalRibbonCount, y=CBa.PercentofInput, hue="Volume", kind = "kde", fill = True).set(title = "CBa Distributions")
sns.displot(CombinedRI, x=CBb.TotalRibbonCount, y=CBb.PercentofInput, hue="Volume", kind = "kde", fill = True).set(title = "CBb Distributions")

